# Verileri İçeri Aktarma

In [8]:
import pandas as pd 
import numpy as np

df = pd.read_csv("../Database/Add_Cluster_Feature.csv")
df.head(3)

,Age,Gender,Air Pollution,Alcohol use,Dust Allergy,OccuPational Hazards,Genetic Risk,chronic Lung Disease,Balanced Diet,Obesity,...,Weight Loss,Shortness of Breath,Wheezing,Swallowing Difficulty,Clubbing of Finger Nails,Frequent Cold,Dry Cough,Snoring,Küme,Level
0,33,1,2,4,5,4,3,2,2,4,...,4,2,2,3,1,2,3,4,8,Low
1,17,1,3,1,5,3,4,2,2,2,...,3,7,8,6,2,1,7,2,0,Medium
2,35,1,4,5,6,5,5,4,6,7,...,7,9,2,1,4,6,7,2,2,High


# Ön işleme

Hedef değişkeni label encode ile encode edelim ki numeric bir biçimde kullanalım.


In [9]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

le = LabelEncoder()
df['Level'] = le.fit_transform(df['Level'])

Modellerimizi test edebilmek için train-test split edelim.

In [27]:
from sklearn.model_selection import train_test_split

X = df.drop("Level", axis=1)
y = df[['Level']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

Verileri standartlaştırıp numpy array biçimine çevirelim ki scikit-learn kütüphanesinden daha fazla verim elde edelim.

In [28]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [12]:
display(X_train[:1]);display(y_train[:5])

array([[-0.76106299,  1.26722657, -1.40072639,  0.57650671,  0.95571931,
         0.11008029, -0.72321282, -1.24521517,  0.7362452 , -1.15240706,
        -0.37188277, -0.49997736, -1.0537369 , -1.16960182, -0.35268247,
        -0.36994787,  1.20890297,  1.6368661 ,  0.14955877,  1.73714076,
         1.89674735,  1.53255725,  1.37188039, -1.36481059]])

array([2, 0, 2, 2, 1])

# Modelleme

Bu aşamada hangi modelleri kullanmamız gerektiğine ilişkin bir inceleme yapmamız gerekiyor. Örneğin bu bir sınıflandırma problemi ancak sınıflandırma problemi diye direkt gidip `DecisionTreeClassfier` modelini kullanmamalıyız. Kullanmayı düşündüğümüz modellerin avantajları ve dezavantajlarını düşünmeliyiz.

| Model İsmi | Modelin Önerilme Nedeni | Avantajları | Dezavantajları |
|--------------|-------------------------------|--------------|------------------|
| Lojistik Regresyon | Lojistik regresyon, sınıflandırma problemlerinde sıklıkla kullanılan istatistiksel bir modeldir. Bu model, bağımlı değişkenin olasılık dağılımını tahmin etmek için kullanılır ve sonuçlarını olasılık değerleri olarak verir. | - Basit ve yüksek yorumlanabilirlik: Katsayıların ve etkilerin anlamlılığı kolayca yorumlanabilir. - Hızlı eğitim süreci: Büyük veri setleriyle bile hızlı eğitim sağlar. | - Lineer ilişkileri modelleyebilir, ancak karmaşık ilişkileri ele alamaz. - Çoklu doğrusal bağlantı veya aşırı uyum gibi sorunlarla karşılaşabilir. |
| Naive Bayes | Naive Bayes, olasılığa dayalı bir sınıflandırma modelidir ve Bayes teoremini temel alır. Bağımsız değişkenler arasında güçlü bir bağımsızlık varsayımı yapar. | - Basit ve hızlı: Hesaplama açısından oldukça verimli ve hızlıdır. - İyi performans: Bağımsız değişkenler arasındaki ilişkiler karmaşıksa bile iyi sonuçlar verir. | - Naive varsayım: Bağımsız değişkenler arasındaki gerçek bağımlılıkları göz ardı eder. Bu, bazı durumlarda gerçek dünyada kötü performansa neden olabilir. - Veri dengesizliği durumlarında yanlı sonuçlara neden olabilir. |
| Destek Vektör Makineleri (SVM) | SVM, bir sınıflandırma modelidir ve özellikle doğrusal ve doğrusal olmayan veri kümeleri arasında ayrım yapmak için etkilidir. Margin maksimizasyonu prensibi üzerine kuruludur. | - Çok yönlü: Doğrusal ve doğrusal olmayan veri kümeleri arasında ayrım yapabilir. - Overfitting'e karşı dirençli: Margin maksimizasyonu, overfitting riskini azaltır. | - Hiperparametre ayarının önemi: Modelin performansı, doğru hiperparametre ayarı gerektirir. - Büyük veri kümeleriyle çalışırken hesaplama açısından yoğun olabilir. |
| Karar Ağaçları (Decision Trees) | Karar ağaçları, veri kümesindeki ilişkileri hiyerarşik olarak gösteren ve sınıflandırma veya regresyon yapmak için kullanılan bir modeldir. | - Yüksek yorumlanabilirlik: Karar ağaçları, sınıflandırma sürecini adım adım açıklayarak yüksek derecede yorumlanabilir sonuçlar sağlar. - Doğrusal olmayan ilişkileri modelleme yeteneği: Doğrusal olmayan veri yapılarını modellemek için kullanılabilirler. | - Aşırı uyum (overfitting): Karmaşık ağaç yapıları, eğitim verilerine aşırı uyum sağlayabilir ve genelleme performansını düşürebilir. - Hassas veri dengesizliği durumlarında zayıf performans gösterebilir. |
| K-En Yakın Komşu (K-Nearest Neighbors) | K-En Yakın Komşu, sınıflandırma ve regresyon problemlerinde kullanılan basit bir modeldir. Bir veri noktasının etrafındaki k-en yakın komşuyu belirler ve çoğunluk sınıfını veya ortalama değeri tahmin eder. | - Basit ve doğal bir yaklaşım: Temelinde sezgisel bir mantık vardır ve karmaşık bir model yapısı gerektirmez. - Esneklik: Doğrusal olmayan ilişkileri modellemek için kullanılabilir. | - Hiperparametre seçimi: Doğru k değerinin seçimi modelin performansını etkiler. - Hesaplama maliyeti: Tahmin yaparken tüm eğitim verileriyle karşılaştırma gereksinimi, büyük veri kümeleri için yüksek hesaplama maliyetine yol açabilir. |

Tüm bu raporlama sonrasında modelleri default olarak (*herhangi bir hiperparametre ayarı ile oynamadan*) deneyip en iyi sonucu veren overfit edilmemiş modeli seçeceğiz.

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Naive Bayes': GaussianNB(),
    'SVM': SVC(random_state=42),
    'Decision Trees': DecisionTreeClassifier(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier()
}

# Model Değerlendirme

Modelleri değerlendirirken birden fazla metrik seçmeliyiz ki pek çok açıdan modelleri değerlendirebilmiş olabilelim. Bunun için `Accuracy Score`, `Precision Score` ve `Recall Score` metriklerini kullanabiliriz.

| Metrik | Açıklama | Beklenti 🔃 | Not ⛔️ |
|:---------:|--------------|:-------------:|:------|
| Accuracy Score | Accuracy, modelin doğru tahmin ettiği gözlemlerin oranını gösterir. | ⬆️ Yüksek - Referans Aralığı: [0, 1] | Dengesiz veri setleri için dikkatli kullanılmalıdır. |
| Precision Score | Precision, modelin pozitif olarak tahmin ettiği örneklerin ne kadarının gerçekten pozitif olduğunu gösterir. | ⬆️ Yüksek - Referans Aralığı: [0, 1] | Düşük precision, yanlış pozitiflerin yüksek olduğunu ve modelin spesifik olmadığını gösterebilir. |
| Recall Score | Recall, gerçek pozitiflerin ne kadarının doğru şekilde tahmin edildiğini gösterir. | ⬆️ Yüksek - Referans Aralığı: [0, 1] | Düşük recall, yanlış negatiflerin yüksek olduğunu ve modelin hassas olmadığını gösterebilir. |


In [30]:
import numpy as np
import plotly.graph_objs as go
from sklearn.metrics import accuracy_score, precision_score, recall_score

def evaluate_model(model, model_name, X_train, X_test, y_train, y_test):
    '''"evaluate_model" işlevi bir makine öğrenimi modelini eğitir, eğitim ve test kümelerindeki
    performansını değerlendirir, doğruluğu, kesinliği ve geri çağırma puanlarını hesaplar ve farklı
    eğitim kümesi boyutlarına göre doğruluk puanlarını çizerek aşırı uyumu görselleştirir.
    
    Parameters
    ----------
    model
        "Model" parametresi, sınıflandırıcı veya regresör gibi değerlendirmek istediğiniz makine öğrenimi
    modelidir.
    model_name
        "model_name" parametresi, değerlendirilmekte olan makine öğrenimi modelinin adını veya
    tanımlayıcısını temsil eden bir dizedir. Analiz edilen belirli modeli tanımlamak için raporlama ve
    görselleştirme amacıyla kullanılır.
    X_train
        X_train, modeli eğitmek için kullanılan tüm giriş özelliklerini içeren eğitim setinin özellik
    matrisidir. Her satırın bir örneği temsil ettiği ve her sütunun bir özelliği temsil ettiği 2 boyutlu
    bir dizidir.
    X_test
        X_test, tahminlerde bulunmak için kullanılan giriş değişkenleri olan test seti özellikleridir.
    Modelin eğitim sırasında görmediği veri kümesinin bir alt kümesidir ve modelin görünmeyen veriler
    üzerindeki performansını değerlendirmek için kullanılır.
    y_train
        'y_train' eğitim seti için hedef değişkendir. Modelin "X_train"deki özelliklere dayalı olarak
    tahmin etmeye çalışacağı gerçek değerleri içerir.
    y_test
        'evaluate_model' işlevindeki 'y_test' parametresi, test kümesinin gerçek etiketlerini temsil eder.
    Tahmin edilen değerleri test setindeki gerçek değerlerle karşılaştırarak modelin performansını
    değerlendirmek için kullanılır. Bu parametre doğruluk, kesinlik ve geri çağırma gibi ölçümlerin
    hesaplanması için gereklidir.
    
    '''
    # Modeli eğit
    model.fit(X_train, y_train)
    
    # Train ve test setleri üzerinde tahmin yap
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Accuracy, precision ve recall hesapla
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    precision = precision_score(y_test, y_test_pred, average='macro')
    recall = recall_score(y_test, y_test_pred, average='macro')
    
    # Sonuçları raporla
    print("Train Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    
    # Overfit olup olmadığını görselleştir
    train_sizes = np.arange(0.1, 1.1, 0.1)
    train_scores = []
    test_scores = []
    
    for size in train_sizes:
        # Eğitim setinin belirli bir yüzdesiyle modeli eğit
        subset_size = int(size * len(X_train))
        X_subset_train, y_subset_train = X_train[:subset_size], y_train[:subset_size]
        model.fit(X_subset_train, y_subset_train)
        
        # Eğitim ve test setleri üzerinde tahmin yap
        train_pred = model.predict(X_train)
        test_pred = model.predict(X_test)
        
        # Accuracy'yi hesapla ve listeye ekle
        train_score = accuracy_score(y_train, train_pred)
        test_score = accuracy_score(y_test, test_pred)
        train_scores.append(train_score)
        test_scores.append(test_score)
    
    # Grafik oluştur
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=train_sizes, y=train_scores, mode='lines+markers', name='Train Accuracy'))
    fig.add_trace(go.Scatter(x=train_sizes, y=test_scores, mode='lines+markers', name='Test Accuracy'))
    fig.update_layout(title=f'Model: {model_name} Performans ve Overfitting',
                      xaxis_title='Eğitim Seti Büyüklüğü',
                      yaxis_title='Accuracy',
                      legend=dict(x=0, y=1, traceorder='normal'))
    fig.show()

In [31]:
import time 

for name, model in models.items():
    start = time.time()
    
    print("Model:", name)
    evaluate_model(model, name, X_train, X_test, y_train, y_test)
    
    end = time.time()
    elapsed_time = end - start
    print(f"Değerlendirme Süresi: {int(elapsed_time//60)}:{int(elapsed_time%60)}")
    print("="*100, "\n\n")

Model: Logistic Regression
Train Accuracy: 1.0
Test Accuracy: 1.0
Precision: 1.0
Recall: 1.0


Değerlendirme Süresi: 0:0


Model: Naive Bayes
Train Accuracy: 0.88875
Test Accuracy: 0.895
Precision: 0.9076568592697626
Recall: 0.8864099766538791


Değerlendirme Süresi: 0:0


Model: SVM
Train Accuracy: 1.0
Test Accuracy: 1.0
Precision: 1.0
Recall: 1.0


Değerlendirme Süresi: 0:0


Model: Decision Trees
Train Accuracy: 1.0
Test Accuracy: 1.0
Precision: 1.0
Recall: 1.0


Değerlendirme Süresi: 0:0


Model: K-Nearest Neighbors
Train Accuracy: 1.0
Test Accuracy: 1.0
Precision: 1.0
Recall: 1.0


Değerlendirme Süresi: 0:0




# Sonuç

Denediğimiz her model için yüksek başarıya sahip olduğunu söyleyebiliriz. Sebepleri konusunda bağımsız değişkenlerimizin hedef değişkenin tahmininde fazlaca yeterli olması, bağımlı değişkeni yüksek oranda açıklaması gibi şeyler söylenebilir. Ancak nihayetinde eğitim ve tahmin süresi en kısa olan herhangi bir modelin kullanılabileceği söylenebilir. 

⛔️ Ayrıca grafiklerden yapılabilecek çıkarıma göre de hiçbir modelimiz overfit (aşırı uyum/ezberleme) olmamıştır.

In [33]:
logistic_regressor = LogisticRegression(random_state=42)
logistic_regressor.fit(X_train, y_train)

import pickle

# export the model to a file
with open('../Models/logistic_regressor_model.pkl', 'wb') as file:
    pickle.dump(logistic_regressor, file)